In [1]:
import time

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.subplots

import numpy as np
import matplotlib.pyplot as plt
import dpx_control_hw as dch

## Settings

In [2]:
CONFIG = '../config.conf'

## Find port and connect

In [3]:
port = dch.find_port()
if port is None:
    port = '/dev/ttyACM0'

dpx = dch.Dosepix(
    port_name=port,
    config_fn=CONFIG,
    thl_calib_fn=None,
    params_fn=None,
    bin_edges_fn=None
)

In [4]:
dpx.dpm.select_adc(analog_out='v_tha')

OMR set to: 381fc0


In [5]:
app = JupyterDash(__name__, update_title=None)
app.layout = html.Div(
    [
        dcc.Graph(id='live-update-graph'),
        dcc.Interval(
            id="interval",
            interval=100,
            n_intervals=0
        )
    ]
)

start_time = time.time()
data = {'time': [], 'bias': [], 'dpx': []}

# Update plot
@app.callback(Output('live-update-graph', 'figure'),
[Input('interval', 'n_intervals')])
def update_data(n):
    try:
        data['bias'].append( int(dpx.dpf.read_bias(), 16) )
        data['dpx'].append( int(dpx.dpf.read_adc(), 16) )
        data['time'].append( time.time() - start_time )
    except ValueError:
        pass

    fig = plotly.subplots.make_subplots(2, 1, vertical_spacing=0.1)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    fig.append_trace({
        'x': data['time'],
        'y': data['dpx'],
        'name': 'DPX ADC',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 1, 1)
    fig.append_trace({
        'x': data['time'],
        'y': data['bias'],
        'name': 'Bias Voltage',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 2, 1)
    return fig

## Run server

In [6]:
app.run_server(debug=True, mode='inline')